## Необходимые пакеты

In [1]:
! pip install --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

  Cloning https://github.com/evfro/polara.git (to revision develop) to /tmp/pip-install-kfb916jx/polara_6fb1b9a85eab4ec086d174b027dce75d
  Running command git clone --filter=blob:none --quiet https://github.com/evfro/polara.git /tmp/pip-install-kfb916jx/polara_6fb1b9a85eab4ec086d174b027dce75d
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/evfro/polara.git to commit 8e48cfd88e616ca53f8bbda1702a3e2c8abaf38e
  Preparing metadata (setup.py) ... done


In [2]:
! wget https://raw.githubusercontent.com/Personalization-Technologies-Lab/RecSys-Course-HSE-Fall23/main/Seminar4/dataprep.py -O dataprep.py
! wget https://raw.githubusercontent.com/Personalization-Technologies-Lab/RecSys-Course-HSE-Fall23/main/Seminar4/evaluation.py -O evaluation.py

--2023-12-24 21:10:38--  https://raw.githubusercontent.com/Personalization-Technologies-Lab/RecSys-Course-HSE-Fall23/main/Seminar4/dataprep.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6589 (6.4K) [text/plain]
Saving to: ‘dataprep.py’

dataprep.py         100%[===================>]   6.43K  --.-KB/s    in 0s      

2023-12-24 21:10:38 (50.9 MB/s) - ‘dataprep.py’ saved [6589/6589]

--2023-12-24 21:10:38--  https://raw.githubusercontent.com/Personalization-Technologies-Lab/RecSys-Course-HSE-Fall23/main/Seminar4/evaluation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP req

In [3]:
pip install similaripy

In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style='white', context='paper')
%config InlineBackend.figure_format = "svg"
import similaripy as sim

from polara import get_movielens_data
from polara.evaluation.pipelines import find_optimal_config, random_grid

from dataprep import transform_indices, leave_last_out, verify_time_split, reindex_data
from evaluation import topn_recommendations, model_evaluate, downvote_seen_items
from scipy.sparse import csr_matrix, diags
from scipy.sparse import coo_matrix
from sklearn.preprocessing import LabelEncoder

In [5]:
def generate_interactions_matrix(data, data_description, rebase_users=False):
    '''
    Convert pandas dataframe with interactions into a sparse matrix.
    Allows reindexing user ids, which help ensure data consistency
    at the scoring stage (assumes user ids are sorted in scoring array).
    '''
    n_users = data_description['n_users']
    n_items = data_description['n_items']
    # get indices of observed data
    user_idx = data[data_description['users']].values # type your code here
    if rebase_users:
        user_idx, user_index = pd.factorize(user_idx, sort=True)
        n_users = len(user_index)
    item_idx = data[data_description['items']].values # type your code here
    feedback = data[data_description['feedback']].values # type your code here
    #print(len(user_idx), len(item_idx))
    # construct rating matrix
    return csr_matrix((feedback, (user_idx, item_idx)), shape=(n_users, n_items))

## Подгрузка данных

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path_to_data = '/content/drive/MyDrive/Recsys_Challenge/'

%cd $path_to_data

/content/drive/MyDrive/Recsys_Challenge


In [8]:
df_clicks_train = pd.read_parquet('df_clicks_train')
df_carts_train = pd.read_parquet('df_carts_train')
df_orders_train = pd.read_parquet('df_orders_train')

In [9]:
## Working with train data
clicks_session_encoder = LabelEncoder()
clicks_aid_encoder = LabelEncoder()

df_clicks_train['session_encoded'] = clicks_session_encoder.fit_transform(df_clicks_train['session'])
df_clicks_train['aid_encoded'] = clicks_aid_encoder.fit_transform(df_clicks_train['aid'])

carts_session_encoder = LabelEncoder()
carts_aid_encoder = LabelEncoder()

df_carts_train['session_encoded'] = carts_session_encoder.fit_transform(df_carts_train['session'])
df_carts_train['aid_encoded'] = carts_aid_encoder.fit_transform(df_carts_train['aid'])

orders_session_encoder = LabelEncoder()
orders_aid_encoder = LabelEncoder()

df_orders_train['session_encoded'] = orders_session_encoder.fit_transform(df_orders_train['session'])
df_orders_train['aid_encoded'] = orders_aid_encoder.fit_transform(df_orders_train['aid'])

## Предобработка данных

In [11]:
from scipy.sparse import coo_matrix

In [12]:
data_description = dict(
    users = 'session_encoded',
    items = 'aid_encoded',
    feedback = 'rating',
    n_users = df_clicks_train['session_encoded'].nunique(),
    n_items = df_clicks_train['aid_encoded'].nunique()
)
data_description

{'users': 'session_encoded',
 'items': 'aid_encoded',
 'feedback': 'rating',
 'n_users': 385470,
 'n_items': 774068}

In [13]:
urm = generate_interactions_matrix(df_clicks_train, data_description, rebase_users=False)

In [14]:
def user_based_cf(urm, test_urm, topK=30, cutoff=20):
    model = sim.cosine(urm)
    model.setdiag(0)
    print(model.nnz)
    #model = model.transpose().tocsr()
    print(model.shape)
    print(test_urm.shape)
    test_sessions = np.unique(test_urm.nonzero()[0])
    scores = sim.dot_product(model, test_urm, target_rows=test_sessions) # filter_cols=test_urm)
    return scores.tocsr()

In [15]:
user_cf_scores = user_based_cf(urm, urm, topK=2000, cutoff=5)

Done: 100%|██████████| 385470/385470 [00:18<00:00, 20521.11it/s]


29895516
(385470, 385470)
(385470, 774068)


Done: 100%|██████████| 385470/385470 [00:16<00:00, 23687.83it/s]


In [16]:
user_cf_scores.shape

(385470, 774068)

In [17]:
df_clicks_train_unique = df_clicks_train[['session',	'aid', 'ts', 'session_encoded',	'aid_encoded']].drop_duplicates(subset='session_encoded', keep='last')

df_clicks_train_unique = df_clicks_train_unique.reset_index()
df_clicks_train_unique = df_clicks_train_unique.drop(columns=['index'])
df_clicks_train_unique['res_encoded'] = ''

In [18]:
from tqdm import tqdm

In [19]:
test_sessions = np.unique(urm.nonzero()[0])
result_click = []
i = 0
test_scores = user_cf_scores[test_sessions]
for left, right in tqdm(zip(test_scores.indptr[:-1], test_scores.indptr[1:])):
    session_recs = test_scores.indices[left:right]
    session_recs_scores = test_scores.data[left:right]
    # Sort aids by score
    sorted_recs = session_recs[np.argsort(session_recs_scores)[::-1]]
    # Join aids
    df_clicks_train_unique['res_encoded'][i] = sorted_recs.tolist()
    result_click.append(sorted_recs)
    i += 1

0it [00:00, ?it/s]<ipython-input-19-35489baf61be>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clicks_train_unique['res_encoded'][i] = sorted_recs.tolist()
385470it [01:42, 3751.71it/s]


In [20]:
df_clicks_train_unique.to_parquet('df_clicks_train_unique.parquet')

## Carts

In [21]:
def simple_knn(data, data_description):
    urm = generate_interactions_matrix(data, data_description, rebase_users=False)
    user_cf_scores = user_based_cf(urm, urm, topK=2000, cutoff=5)

    df_unique = data[['session',	'aid', 'ts', 'session_encoded',	'aid_encoded']].drop_duplicates(subset='session_encoded', keep='last')
    df_unique = df_unique.reset_index()
    df_unique = df_unique.drop(columns=['index'])
    df_unique['res_encoded'] = ''

    test_sessions = np.unique(urm.nonzero()[0])
    result_click = []
    i = 0
    test_scores = user_cf_scores[test_sessions]
    for left, right in tqdm(zip(test_scores.indptr[:-1], test_scores.indptr[1:])):
        session_recs = test_scores.indices[left:right]
        session_recs_scores = test_scores.data[left:right]
        # Sort aids by score
        sorted_recs = session_recs[np.argsort(session_recs_scores)[::-1]]
        # Join aids
        df_unique['res_encoded'][i] = sorted_recs.tolist()
        result_click.append(sorted_recs)
        i += 1
    return df_unique

In [22]:
data_description = dict(
    users = 'session_encoded',
    items = 'aid_encoded',
    feedback = 'rating',
    n_users = df_carts_train['session_encoded'].nunique(),
    n_items = df_carts_train['aid_encoded'].nunique()
)
data_description

{'users': 'session_encoded',
 'items': 'aid_encoded',
 'feedback': 'rating',
 'n_users': 123891,
 'n_items': 196951}

In [23]:
df_carts_train['rating'] = 1
df_carts_train_unique = simple_knn(df_carts_train, data_description)

Done: 100%|██████████| 123891/123891 [00:00<00:00, 346588.86it/s]


3463612
(123891, 123891)
(123891, 196951)


Done: 100%|██████████| 123891/123891 [00:00<00:00, 180057.29it/s]
0it [00:00, ?it/s]<ipython-input-21-3fb5911632f7>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['res_encoded'][i] = sorted_recs.tolist()
123891it [00:31, 3873.18it/s]


In [24]:
df_carts_train_unique.to_parquet('df_carts_train_unique.parquet')

## Orders

In [25]:
data_description = dict(
    users = 'session_encoded',
    items = 'aid_encoded',
    feedback = 'rating',
    n_users = df_orders_train['session_encoded'].nunique(),
    n_items = df_orders_train['aid_encoded'].nunique()
)
data_description

{'users': 'session_encoded',
 'items': 'aid_encoded',
 'feedback': 'rating',
 'n_users': 55845,
 'n_items': 79824}

In [26]:
df_orders_train['rating'] = 1
df_orders_train_unique = simple_knn(df_orders_train, data_description)

Done: 100%|██████████| 55845/55845 [00:00<00:00, 832081.37it/s]


756440
(55845, 55845)
(55845, 79824)


Done: 100%|██████████| 55845/55845 [00:00<00:00, 463915.44it/s]
0it [00:00, ?it/s]<ipython-input-21-3fb5911632f7>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['res_encoded'][i] = sorted_recs.tolist()
55845it [00:14, 3869.84it/s]


In [27]:
df_orders_train_unique.to_parquet('df_orders_train_unique.parquet')

## KNN + Normalization

## CLICKS

In [28]:
def scoring_func(user_cf_scores, df, column, test_sessions):
    i = 0
    test_scores = user_cf_scores[test_sessions]
    for left, right in tqdm(zip(test_scores.indptr[:-1], test_scores.indptr[1:])):
        session_recs = test_scores.indices[left:right]
        session_recs_scores = test_scores.data[left:right]
        # Sort aids by score
        sorted_recs = session_recs[np.argsort(session_recs_scores)[::-1]]
        # Join aids
        df[column][i] = sorted_recs.tolist()
        result_click.append(sorted_recs)
        i += 1
    return df

In [29]:
def norm_knn(data, data_description):
    urm = generate_interactions_matrix(data, data_description, rebase_users=False)
    urm_tfidf = sim.normalization.tfidf(urm)

    user_cf_scores = user_based_cf(urm, urm, topK=2000, cutoff=5)
    user_cf_scores_tfidf = user_based_cf(urm_tfidf, urm_tfidf, topK=2000, cutoff=5)

    df_unique = data[['session',	'aid', 'ts', 'session_encoded',	'aid_encoded']].drop_duplicates(subset='session_encoded', keep='last')
    df_unique = df_unique.reset_index()
    df_unique = df_unique.drop(columns=['index'])
    df_unique['res_encoded'] = ''
    df_unique['res_encoded_tfidf'] = ''

    test_sessions = np.unique(urm.nonzero()[0])
    result_click = []
    i = 0
    test_scores = user_cf_scores[test_sessions]
    df_unique = scoring_func(user_cf_scores, df_unique, 'res_encoded', test_sessions)
    df_unique = scoring_func(user_cf_scores_tfidf, df_unique, 'res_encoded_tfidf', test_sessions)
    return df_unique

In [30]:
data_description = dict(
    users = 'session_encoded',
    items = 'aid_encoded',
    feedback = 'rating',
    n_users = df_clicks_train['session_encoded'].nunique(),
    n_items = df_clicks_train['aid_encoded'].nunique()
)
data_description

{'users': 'session_encoded',
 'items': 'aid_encoded',
 'feedback': 'rating',
 'n_users': 385470,
 'n_items': 774068}

In [31]:
df_clicks_train['rating'] = 1
df_clicks_train_unique = norm_knn(df_clicks_train, data_description)

Done: 100%|██████████| 385470/385470 [00:12<00:00, 31948.50it/s]


29895516
(385470, 385470)
(385470, 774068)


Done: 100%|██████████| 385470/385470 [00:10<00:00, 37038.77it/s]


29895516
(385470, 385470)
(385470, 774068)


Done: 100%|██████████| 385470/385470 [00:12<00:00, 29855.33it/s]
0it [00:00, ?it/s]<ipython-input-28-0a6b921e72e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = sorted_recs.tolist()
385470it [01:46, 3624.40it/s]
385470it [01:49, 3506.55it/s]


In [32]:
df_clicks_train_unique.head()

,session,aid,ts,session_encoded,aid_encoded,res_encoded,res_encoded_tfidf
0,0,1582117,1660244973157,0,660071,"[756623, 447279, 767847, 309654, 278368, 51127...","[756623, 767847, 278368, 511274, 295778, 30965..."
1,1,959544,1659991199405,1,400102,"[723568, 44170, 299744, 89661, 365911, 742127,...","[723568, 89661, 44170, 299744, 356498, 20843, ..."
2,3,54857,1660075780356,2,22877,"[257579, 410617, 22877, 560279, 592905, 3539, ...","[410617, 22877, 257579, 408573, 754463, 240916..."
3,6,1329892,1659996712947,3,554595,"[102928, 554595, 599982, 404320, 362899, 77393...","[102928, 554595, 404320, 599982, 44850, 773930..."
4,7,129869,1660222041409,4,54267,"[54267, 39989, 231436, 544107, 353385, 624719,...","[54267, 231436, 39989, 544107, 250768, 292773,..."


In [33]:
df_clicks_train_unique.to_parquet('df_clicks_train_unique.parquet')

## Carts

In [34]:
data_description = dict(
    users = 'session_encoded',
    items = 'aid_encoded',
    feedback = 'rating',
    n_users = df_carts_train['session_encoded'].nunique(),
    n_items = df_carts_train['aid_encoded'].nunique()
)
data_description

{'users': 'session_encoded',
 'items': 'aid_encoded',
 'feedback': 'rating',
 'n_users': 123891,
 'n_items': 196951}

In [35]:
df_carts_train['rating'] = 1
df_carts_train_unique = norm_knn(df_carts_train, data_description)

Done: 100%|██████████| 123891/123891 [00:00<00:00, 440086.48it/s]


3463612
(123891, 123891)
(123891, 196951)


Done: 100%|██████████| 123891/123891 [00:00<00:00, 287481.77it/s]


3463612
(123891, 123891)
(123891, 196951)


Done: 100%|██████████| 123891/123891 [00:00<00:00, 164205.03it/s]
0it [00:00, ?it/s]<ipython-input-28-0a6b921e72e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = sorted_recs.tolist()
123891it [00:31, 3880.82it/s]
123891it [00:32, 3833.09it/s]


In [36]:
df_carts_train_unique.to_parquet('df_carts_train_unique.parquet')

In [37]:
df_carts_train_unique

,session,aid,ts,session_encoded,aid_encoded,res_encoded,res_encoded_tfidf
0,1,105393,1659991168139,0,11165,"[11165, 22654, 113492, 90229, 68176, 83514, 11...","[11165, 22654, 113492, 90229, 110168, 83514, 6..."
1,3,812246,1659997178730,1,85682,"[104042, 3076, 75342, 78965, 176303, 13517, 59...","[104042, 75342, 3076, 13517, 133579, 5823, 789..."
2,11,1528062,1660209891347,2,162083,"[121198, 11935, 36190, 145931, 127393, 90991, ...","[121198, 127393, 11935, 145931, 90991, 21238, ..."
3,13,139863,1659735763747,3,14763,[],[]
4,14,200631,1660049357293,4,21059,"[21059, 98067, 143545, 56457, 142407, 21345, 1...","[21059, 56457, 38408, 143545, 170858, 98067, 1..."
...,...,...,...,...,...,...,...
123886,327655,1485859,1660053679181,60632,157612,"[93002, 104213, 58377, 61542, 65396, 128869, 1...","[93002, 104213, 61542, 128869, 28828, 98547, 1..."
123887,327658,193755,1660069318178,60633,20378,"[116290, 149467]","[116290, 149467]"
123888,327664,111057,1659878447598,60634,11732,"[168759, 122276, 84033, 79595, 72438, 70721, 1...","[168759, 161348, 122276, 13910, 142707, 98354,..."
123889,327675,933444,1660067122781,60635,98531,"[6490, 15768, 168058, 74564, 191476, 149684, 7...","[15768, 6490, 168058, 191476, 74564, 43597, 63..."


## Orders

In [46]:
data_description = dict(
    users = 'session_encoded',
    items = 'aid_encoded',
    feedback = 'rating',
    n_users = df_orders_train['session_encoded'].nunique(),
    n_items = df_orders_train['aid_encoded'].nunique()
)
data_description

{'users': 'session_encoded',
 'items': 'aid_encoded',
 'feedback': 'rating',
 'n_users': 55845,
 'n_items': 79824}

In [47]:
df_orders_train['rating'] = 1
df_orders_train_unique = norm_knn(df_orders_train, data_description)

Done: 100%|██████████| 55845/55845 [00:00<00:00, 812478.00it/s]


756440
(55845, 55845)
(55845, 79824)


Done: 100%|██████████| 55845/55845 [00:00<00:00, 859796.45it/s]


756440
(55845, 55845)
(55845, 79824)


Done: 100%|██████████| 55845/55845 [00:00<00:00, 428240.60it/s]
0it [00:00, ?it/s]<ipython-input-28-0a6b921e72e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = sorted_recs.tolist()
55845it [00:14, 3958.54it/s]
55845it [00:16, 3342.64it/s]


In [40]:
df_orders_train_unique.to_parquet('df_orders_train_unique.parquet')

In [51]:
df_orders_train_unique

,session,aid,ts,session_encoded,aid_encoded,res_encoded,res_encoded_tfidf,pred,pred_tfidf
0,3,54857,1659999789346,0,2335,[0],[0],[133],[133]
1,11,1145803,1659902394985,1,49003,[],[],[],[]
2,35,1162085,1659788011065,2,49709,"[49709, 17508, 16114, 76864, 49594, 42178, 29192]","[49709, 16114, 17508, 49594, 76864, 29192, 42178]","[1162085, 411718, 381203, 1785754, 1159379, 98...","[1162085, 381203, 411718, 1159379, 1785754, 68..."
3,40,987525,1660302982234,3,42241,[],[],[],[]
4,78,225700,1659984507541,4,9576,"[68964, 64392, 3230, 16220, 51750, 12615, 6555...","[68964, 64392, 47733, 78040, 9576, 23546, 4664...","[1604220, 1497729, 75565, 383437, 1208112, 296...","[1604220, 1497729, 1116346, 1813366, 225700, 5..."
...,...,...,...,...,...,...,...,...,...
55840,327655,1485859,1660053731982,27453,63834,"[2927, 45086, 33277, 50346, 79176, 898, 30746,...","[2927, 69950, 898, 25219, 69980, 45086, 30746,...","[68752, 1053941, 781961, 1176051, 1840622, 210...","[68752, 1626675, 21029, 592506, 1627528, 10539..."
55841,327658,193755,1660069373200,27454,8244,"[37577, 39866, 50171, 1550, 66955, 13809, 7979...","[37577, 39866, 50171, 1550, 66955, 58294, 1299...","[881286, 933576, 1171883, 37199, 1556672, 3255...","[881286, 933576, 1171883, 37199, 1556672, 1358..."
55842,327664,215189,1659791135713,27455,9152,[47034],[47034],[1099405],[1099405]
55843,327675,933444,1660067202654,27456,39857,"[68411, 49435, 33941, 32097, 29232, 28562, 567...","[68411, 65414, 39789, 49435, 5677, 57740, 1554...","[1590969, 1156177, 797207, 754314, 687576, 671...","[1590969, 1520892, 931637, 1156177, 131715, 13..."


In [49]:
df_orders_train_unique['pred'] = df_orders_train_unique['res_encoded'].apply(orders_aid_encoder.inverse_transform)

In [50]:
df_orders_train_unique['pred_tfidf'] = df_orders_train_unique['res_encoded_tfidf'].apply(orders_aid_encoder.inverse_transform)

In [ ]:
df_orders_train_unique.to_parquet('df_clicks_pred.parquet')